In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.datasets import mnist

# Genetik algoritma aşamalarını içeren fonksiyonlar

def initialize_population(population_size, num_features):
    population = np.random.randint(2, size=(population_size, num_features))
    return population

def calculate_fitness(X, y, selected_features):
    X_selected = X[:, selected_features]
    clf = RandomForestClassifier(n_estimators=100)
    clf.fit(X_selected, y)
    y_pred = clf.predict(X_selected)
    fitness = accuracy_score(y, y_pred)
    return fitness

def select_parents(population, fitness):
    parents = []
    fitness_sum = np.sum(fitness)
    probabilities = fitness / fitness_sum
    for _ in range(len(population) // 2):
        parent1_index = np.random.choice(len(population), p=probabilities)
        parent2_index = np.random.choice(len(population), p=probabilities)
        parent1 = population[parent1_index]
        parent2 = population[parent2_index]
        parents.append((parent1, parent2))
    return parents

def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, len(parent1) - 1)
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    return child1, child2

def mutate(individual, mutation_rate):
    for i in range(len(individual)):
        if np.random.random() < mutation_rate:
            individual[i] = 1 - individual[i]
    return individual

# MNIST veri kümesini yükle
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Verileri ölçeklendirin
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.reshape(len(x_train), -1))
x_test_scaled = scaler.transform(x_test.reshape(len(x_test), -1))

# Genetik algoritma parametreleri
population_size = 100
num_generations = 5
mutation_rate = 0.1

# Sınıflandırma modeli tanımı
clf = RandomForestClassifier(n_estimators=100)

# Veri seti boyutları ve özellik sayısı
num_samples, num_features = x_train_scaled.shape

# Başlangıç popülasyonunu oluştur
population = initialize_population(population_size, num_features)

# Genetik algoritma döngüsü
for generation in range(num_generations):
    # Uygunluk değerlerini hesapla
    fitness = np.array([calculate_fitness(x_train_scaled, y_train, selected_features) for selected_features in population])
    
    # En iyi bireyi seç
    best_individual = population[np.argmax(fitness)]
    best_fitness = np.max(fitness)
    
    # Seçilen özellikleri uygula
    selected_features = np.where(best_individual == 1)[0]
    x_train_selected = x_train_scaled[:, selected_features]
    x_test_selected = x_test_scaled[:, selected_features]
    
    # Sınıflandırma modelini eğit
    clf.fit(x_train_selected, y_train)
    
    # Test veri seti üzerinde sınıflandırma yap
    y_pred = clf.predict(x_test_selected)
    
    # Doğruluk değerini hesapla
    accuracy = accuracy_score(y_test, y_pred)
    
    # Sonuçları yazdır
    print(f"Generation {generation+1}: Best Fitness = {best_fitness}, Accuracy = {accuracy}")
    
    # Ebeveynleri seç
    parents = select_parents(population, fitness)
    
    # Yeni nesil oluştur
    offspring = []
    for parent1, parent2 in parents:
        child1, child2 = crossover(parent1, parent2)
        child1 = mutate(child1, mutation_rate)
        child2 = mutate(child2, mutation_rate)
        offspring.append(child1)
        offspring.append(child2)
    population = np.array(offspring)

# En iyi bireyi seç
best_individual = population[np.argmax(fitness)]
best_features = np.where(best_individual == 1)[0]
x_train_selected = x_train_scaled[:, best_features]
x_test_selected = x_test_scaled[:, best_features]

# Sınıflandırma modelini eğit
clf.fit(x_train_selected, y_train)

# Test veri seti üzerinde sınıflandırma yap
y_pred = clf.predict(x_test_selected)
# Doğruluk değerini hesapla
accuracy = accuracy_score(y_test, y_pred)

# Sonuçları yazdır
print(f"Final Result: Best Fitness = {best_fitness}, Accuracy = {accuracy}")


11490434/11490434 [==============================] - 2s 0us/step
Generation 1: Best Fitness = 0.11236666666666667, Accuracy = 0.9651
Generation 2: Best Fitness = 0.11236666666666667, Accuracy = 0.9651
Generation 3: Best Fitness = 0.11236666666666667, Accuracy = 0.9669
Generation 4: Best Fitness = 0.11236666666666667, Accuracy = 0.9636
Generation 5: Best Fitness = 0.11236666666666667, Accuracy = 0.9637
Final Result: Best Fitness = 0.11236666666666667, Accuracy = 0.9665
